In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Define Functions

In [5]:
## SCC functions
from SCC_functions import *

ModuleNotFoundError: No module named 'SCC_functions'

In [ ]:
## MACC functions and parameters

# default settings
P_50 = 100  # 187.5 * 0.7625000000000001
P_100 = 300  # 187.5
s = 0.05
r = 0.04

def abatement(P, P0, P_50=P_50, r=r, s=s, P_100=P_100):  # 0.05
    if P0 >= P_50:
        print("P0 is greater than P_50")
    elif ((P_100-P0)/(P_50-P0))**(s/r) <= 2:
        print("MAC curve condition not satisfied")
    
    P_h = P0 + ((P_50 - P0) ** (-s / r) - 2 * (P_100 - P0) ** (-s / r)) ** (-r / s)
    Am = 1 + ((P_100 - P0) / (P_h - P0)) ** (-s / r)
    
    A = Am / (1 + ((P - P0) / (P_h - P0)) ** (-s / r))
    
    return A

## calculate emissions from forecasted abatement and CO2 baseline

def abatement_to_emissions(forecasted_abatement, CO2_baseline):
    CO2_emissions = CO2_baseline*(1-forecasted_abatement)
    
    return CO2_emissions

# Steps

## 0. Years and Initial Temperature Anomaly

In [ ]:
start_year = 1750
last_historical_year = 2020
end_year = 3000

years = create_years_array(start_year, end_year)
num_of_years = len(years)

In [ ]:
## years to calculate SCC
first_perturbed_year = last_historical_year
last_perturbed_year = 2500
year_of_perturbation = create_years_array(first_perturbed_year, last_perturbed_year)

In [ ]:
size_of_perturbation = 1

## 1. Create Unperturbed Temperature Profile

- use historical emissions plus added on shape

In [ ]:
test_emissions = pd.read_csv("ssp245_CO2.csv", index_col=0, nrows=last_historical_year-start_year+1)
# test_emissions = pd.concat([test_emissions],keys=['ssp245'],axis=1)

In [ ]:
def perturb_co2(df,year):
    df = df.copy()
    df.loc[year,'carbon_dioxide'] += size_of_perturbation * 12/44 # perturbation of CO2 in GtC
    return df
# def perturb_gas(df,year,gas):
#     df = df.copy()
#     df.loc[year,gas] += 1 # perturbation of 1 GtCH4 (maybe it's MtCH4)
#     return df

# perturbed_emms = pd.concat([ssp_emms]+[perturb_co2(ssp_emms,SCC_year)],keys=['ssp245']+['ssp245-p'+str(SCC_year)],axis=1)
# perturbed_emms = pd.concat([ssp_emms]+[perturb_co2(ssp_emms,year) for year in np.arange(1845,2101)],keys=['ssp245']+['ssp245-p'+str(year) for year in np.arange(1845,2101)],axis=1)
# perturbed_emms_meth = pd.concat([ssp_emms]+[perturb_gas(ssp_emms,year,'methane') for year in np.arange(1845,2101)],keys=['ssp245']+['ssp245-p'+str(year) for year in np.arange(1845,2101)],axis=1)

In [ ]:
test_emissions

In [ ]:
plt.plot(test_emissions)

In [ ]:
for i in range(end_year-last_historical_year):
#     if 12 - (i-2000) > 0:
#         test_emissions.loc[last_historical_year+i+1] = 12 - (i-2000)
#     else:
    test_emissions.loc[last_historical_year+i+1] = 0
#     test_emissions.loc[last_historical_year, 'Test']

In [ ]:
test_emissions.loc[last_historical_year]

In [ ]:
plt.plot(test_emissions)

In [ ]:
test_emissions

In [ ]:
def perturb_co2(df,year):
    df = df.copy()
    df.loc[year,'carbon_dioxide'] += size_of_perturbation * 12/44 # perturbation of CO2 in GtC
    return df
# def perturb_gas(df,year,gas):
#     df = df.copy()
#     df.loc[year,gas] += 1 # perturbation of 1 GtCH4 (maybe it's MtCH4)
#     return df

# perturbed_emms = pd.concat([ssp_emms]+[perturb_co2(ssp_emms,SCC_year)],keys=['ssp245']+['ssp245-p'+str(SCC_year)],axis=1)
perturbed_emms = pd.concat([test_emissions]+[perturb_co2(test_emissions,year) for year in year_of_perturbation],keys=['ssp245']+['ssp245-p'+str(year) for year in year_of_perturbation],axis=1)

In [ ]:
# Generate a default parameter dataframe:

test_gas_parameters = get_gas_parameter_defaults()

test_thermal_parameters = get_thermal_parameter_defaults()

In [ ]:
test_gas_parameters = test_gas_parameters.reindex(perturbed_emms.columns.levels[1],axis=1,level=1)

In [ ]:
perturbed_emms

In [ ]:
# pulse_run = run_FaIR(emissions_in=test_emissions,gas_parameters=test_gas_parameters,thermal_parameters=test_thermal_parameters)

In [ ]:
# plt.plot(pulse_run['T'])

In [ ]:
# pulse_run['T'].idxmax()

In [ ]:
pulse_run = run_FaIR(emissions_in=perturbed_emms,gas_parameters=test_gas_parameters,thermal_parameters=test_thermal_parameters)

In [ ]:
plt.plot(pulse_run['T']['ssp245'])

In [ ]:
T = pulse_run['T']['ssp245']['default'].tolist()

# T_perturbed = pulse_run['T']['ssp245-p'+str(SCC_year)]['default'].tolist()

In [ ]:
pulse_run

## Total Consumption

In [ ]:
consumption_growth = 0.03

In [ ]:
W = create_total_consumption(years, W_fixed_year=2020, W_fixed=80, consumption_growth=consumption_growth)

In [ ]:
plt.plot(years, W)
plt.xlabel("Year")
plt.ylabel("Absolute Consumption ($trillion/year)")
# plt.xlim(2000, 2100)
# plt.ylim(0, 400)
plt.xlim(2000, 2500)
plt.ylim(10, 100000)
plt.yscale("log")

alternative functional form for above for the fast transition

## 2. Calculate SCC as a function of time for a range of years

In [ ]:
T_TCRE_1 = 0.00054  # need to check this for FAIR!
k_s = 0.12

In [ ]:
gamma = 2
D0 = 0.00267

In [ ]:
# consumption_discount = 0.035
consumption_discount = consumption_growth + 0.015

In [ ]:
# consumption_discount = 0.035
# consumption_discount = consumption_growth + 0.015

SCC_list = []

for j in range(len(year_of_perturbation)):
#     T_perturbed = create_geometric_T_perturbed(years, T=T, SCC_year=year_of_perturbation[j], T_TCRE=T_TCRE_1*size_of_perturbation, k_s=k_s)   
    T_perturbed = pulse_run['T']['ssp245-p'+str(year_of_perturbation[j])]['default'].tolist()
    discount_function = create_discount_function(years, year_of_perturbation[j], consumption_discount=consumption_discount)
    
    time_series_data = {'years': years, 'W': W, 'discount function': discount_function, 'T': T, 'T perturbed': T_perturbed}
#     print(len(time_series_data['T']))
    time_series_df = pd.DataFrame(data=time_series_data).set_index('years')

    SCC = SCC_calculator(time_series_df, size_of_perturbation=size_of_perturbation, gamma=gamma, D0=D0)
    
    if j  == 0:
        print(SCC)
#         print(time_series_df)

    if SCC < P_100:
        SCC_list.append(SCC)
    else:
        SCC_list.append(P_100)
        break

In [ ]:
SCC_array = np.asarray(SCC_list)  # convert to numpy array

In [ ]:
plt.plot(year_of_perturbation[:len(SCC_array)], SCC_array)
plt.xlabel("Year")
plt.ylabel("SCC ($)")

### Check 100% abatement has been achieved!

In [ ]:
if SCC_array[-1] < P_100:
    print("P_100 not achieved by achieved by final perturbed year")

In [ ]:
SCC_forecasted = []

for i in range(num_of_years):
    if i < len(SCC_array):
        SCC_forecasted.append(SCC_array[i])
    else:
#         SCC_forecasted.append(SCC_array[-1])
        SCC_forecasted.append(P_100)

SCC_forecasted = np.array(SCC_forecasted)

In [ ]:
# SCC_forecasted

In [ ]:
plt.plot(years, SCC_forecasted)
plt.xlabel("Year")
plt.ylabel("SCC ($)")
plt.yscale("log")

## 3. Abatement as a function of time

In [ ]:
P0 = SCC_forecasted[0]

In [ ]:
forecasted_abatement = abatement(P=SCC_forecasted, P0=P0)  # , r=consumption_discount

problems with certain values due to constraints of the model

In [ ]:
plt.plot(years, forecasted_abatement)
plt.xlabel("Year")
plt.ylabel("Abatement")

## 4. Abatement to emissions

In [ ]:
CO2_baseline = 40

In [ ]:
CO2_baseline

In [ ]:
forecasted_emissions = abatement_to_emissions(forecasted_abatement, CO2_baseline)

In [ ]:
# plt.plot(ssp245_CO2_past, label="historical emissions")
plt.plot(years, forecasted_emissions, label="forecasted emissions")
plt.xlabel("Year")
plt.ylabel("CO2 Emissions / GtCO2")
plt.legend()

## 5. Cumulative Emissions to Temperature Change

In [ ]:
cumulative_emissions_array = np.append(np.zeros(1), np.cumsum(forecasted_emissions)[:-1])

In [ ]:
temperature_change = T_TCRE_1*cumulative_emissions_array

In [ ]:
max(temperature_change)

In [ ]:
T_forecast_iteration0 = T_0 + temperature_change
# T_forecast_iteration0_plateau = T_2019 + temperature_change_plateau

In [ ]:
plt.plot(years, T, label="initial guess")
plt.plot(years, T_forecast_iteration0, label="iteration 0")
# plt.plot(T_forecast_years, T_forecast_iteration0_plateau, label="iteration 0 plateau")
plt.xlabel("year")
plt.ylabel("temperature / K")
plt.legend()

# Iterating Further

In [ ]:
# T_forecast_first_loop = T_forecast_iteration0_plateau
# T_forecast_iteration = T_forecast_iteration0_plateau

In [ ]:
T_forecast_first_loop = T_forecast_iteration0
T_forecast_iteration = T_forecast_iteration0

In [ ]:
num_of_iterations = 99

first_perturbed_year = start_year

year_of_perturbation = np.arange(first_perturbed_year, last_perturbed_year+1)

plt.plot(years, T, label="initial guess")
plt.plot(years, T_forecast_first_loop, label="iteration 0")

for k in range(num_of_iterations):
#     T_iteration = np.concatenate([T_gas_df['CO2_best'],T_forecast_iteration[1:]])

    SCC_list = []

    for j in range(len(year_of_perturbation)):

        T_perturbed = create_geometric_T_perturbed(years, T=T_forecast_iteration, SCC_year=year_of_perturbation[j], T_TCRE=T_TCRE_1*size_of_perturbation, k_s=k_s)
        discount_function = create_discount_function(years, year_of_perturbation[j], consumption_discount=consumption_discount)

        time_series_data = {'years': years, 'W': W, 'discount function': discount_function, 'T': T_forecast_iteration, 'T perturbed': T_perturbed}
        time_series_df = pd.DataFrame(data=time_series_data).set_index('years')

        SCC = SCC_calculator(time_series_df, size_of_perturbation=size_of_perturbation, gamma=gamma, D0=D0)

        if j  == 0:
            print(SCC)

        if SCC < P_100:
            SCC_list.append(SCC)
        else:
            SCC_list.append(P_100)
            break

    SCC_array = np.asarray(SCC_list)

    SCC_forecasted = []

    for i in range(num_of_years):
        if i < len(SCC_array):
            SCC_forecasted.append(SCC_array[i])
        else:
            SCC_forecasted.append(P_100)

    SCC_forecasted = np.array(SCC_forecasted)

    P0 = SCC_forecasted[0]

    forecasted_abatement = abatement(P=SCC_forecasted, P0=P0)
    forecasted_emissions = abatement_to_emissions(forecasted_abatement, CO2_baseline)

    cumulative_emissions_array = np.append(np.zeros(1), np.cumsum(forecasted_emissions)[:-1])

    temperature_change = T_TCRE_1*cumulative_emissions_array
    
    T_forecast_iteration = T_0 + temperature_change
    
    plt.plot(years, T_forecast_iteration, label="iteration "+str(k+1))
    
    if k == 0:
        peak_T = max(T_forecast_iteration)
    else:
        previous_peak_T = peak_T
        peak_T = max(T_forecast_iteration)
        if abs(peak_T - previous_peak_T) < 0.005:
            print(k)
            break
    
    if k == num_of_iterations - 1:
        print("convergence condition not achieved")
        print(f"{consumption_discount=}")
        print(f"{consumption_growth=}")
        print(f"{P_50=}")
        print(f"{s=}")
        print(f"{r=}")
        print(f"{P_100=}")

plt.xlabel("Year")
plt.ylabel("Temperature / K")
plt.legend()

***this is fairly analogous to the Newton-Raphson Method***
- the initial temperature profile is essentially just a starting guess; getting pushed toward temperature profile solution
- could stop the iterations when the difference between two consecutive lines is small

In [ ]:
## peak Temperature
max(T_forecast_iteration)

under default settings: 2.435168016568933